# Mission to Mars

#### Web scraping to find the top news of the day

In [1]:
#import dependencies
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from splinter import Browser
import os
import pandas as pd
import time
import pprint

In [2]:
# when using chrome driver
!which chromedriver

/usr/local/bin/chromedriver


In [3]:
# when running on chrome browser
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser("chrome", **executable_path, headless = False)

In [4]:
#visiting the page
url = "https://mars.nasa.gov/news/"
browser.visit(url)

In [5]:
#using bs to write it into html
html = browser.html
soup = bs(html,"html.parser")
time.sleep(2)

In [6]:
#pprint.pprint(soup,depth=1)

In [7]:
news_title = soup.find("div",class_="content_title").text
news_paragraph = soup.find("div", class_="article_teaser_body").text
print(f"Title: {news_title}")
print(f"Para: {news_paragraph}")

Title: What's Mars Solar Conjunction, and Why Does It Matter?
Para: NASA spacecraft at Mars are going to be on their own for a few weeks when the Sun comes between Mars and Earth, interrupting communications.


## JPL Mars Space Images - Featured Image

In [8]:
url_image = "https://www.jpl.nasa.gov/spaceimages/?search=&category=featured#submit"
browser.visit(url_image)
time.sleep(2)

In [9]:
# import dependencies for parsing website to fetch images
from urllib.parse import urlsplit

In [10]:
# splitting website
url_split = urlsplit(url_image)
url_split

SplitResult(scheme='https', netloc='www.jpl.nasa.gov', path='/spaceimages/', query='search=&category=featured', fragment='submit')

In [11]:
#create base url
base_url=url_split.scheme+"://"+url_split.netloc
#print(f"{url_split.scheme}://{url_split.netloc}")
base_url

'https://www.jpl.nasa.gov'

#### finding image path

In [12]:
#Design an xpath selector to grab the image
xpath = "//*[@id=\"page\"]/section[3]/div/ul/li[1]/a/div/div[2]/img"

In [13]:
#Use splinter to click on the mars featured image
#to bring the full resolution image
results = browser.find_by_xpath(xpath)
img = results[0]
img.click()

In [15]:
html_image = browser.html
soup = bs(html_image, "html.parser")

In [16]:
img_url=soup.find("img",class_="fancybox-image")["src"]

In [17]:
# define featured image url
featured_image_url=base_url+"/"+img_url
featured_image_url

'https://www.jpl.nasa.gov//spaceimages/images/largesize/PIA23354_hires.jpg'

### Mars Weather

In [18]:
#get mars weather from twitter feed
url_weather='https://twitter.com/marswxreport?lang=en'
browser.visit(url_weather)

In [19]:
#get weather data
html_weather = browser.html
soup=bs(html_weather,"html.parser")
mars_weather = soup.find("p", class_="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text").text
print(mars_weather)

InSight sol 261 (2019-08-21) low -102.4ºC (-152.4ºF) high -26.6ºC (-15.8ºF)
winds from the SSE at 4.9 m/s (11.0 mph) gusting to 16.0 m/s (35.8 mph)
pressure at 7.70 hPapic.twitter.com/MhPPOHJg3m


### Mars Facts

In [20]:
# url to scrape
url_facts="https://space-facts.com/mars/"

In [21]:
#read html using function .read_html and select second record for display
facts_table=pd.read_html(url_facts)
facts_table[1]

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [22]:
# name the columns and set index
mars_facts = facts_table[1]
mars_facts.columns = ["Parameter", "Values"]
mars_facts.set_index(["Parameter"])

,Values
Parameter,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [23]:
# convert the table to html
mars_html_table=mars_facts.to_html()

In [24]:
# convert the html to multiple line displays
mars_html_table=mars_html_table.replace("\n", "")

In [25]:
#disply the html table
mars_html_table

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Parameter</th>      <th>Values</th>    </tr>  </thead>  <tbody>    <tr>      <th>0</th>      <td>Equatorial Diameter:</td>      <td>6,792 km</td>    </tr>    <tr>      <th>1</th>      <td>Polar Diameter:</td>      <td>6,752 km</td>    </tr>    <tr>      <th>2</th>      <td>Mass:</td>      <td>6.39 × 10^23 kg (0.11 Earths)</td>    </tr>    <tr>      <th>3</th>      <td>Moons:</td>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>4</th>      <td>Orbit Distance:</td>      <td>227,943,824 km (1.38 AU)</td>    </tr>    <tr>      <th>5</th>      <td>Orbit Period:</td>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>6</th>      <td>Surface Temperature:</td>      <td>-87 to -5 °C</td>    </tr>    <tr>      <th>7</th>      <td>First Record:</td>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>8</th>      <td>Recorded By:</td>      <td>Egyptian astronome

### Mars Hemispheres

In [26]:
# url to scrape
url_hem="https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url_hem)

In [27]:
# create bs object and parse
html = browser.html
hemi_soup = bs(html, 'html.parser')
time.sleep(2)

In [30]:
#define blank list
hemisphere_image_urls = []

# Get a List of All the Hemispheres
links = browser.find_by_css("a.product-item h3")
for item in range(len(links)):
    hemisphere = {}
    
    # Find Element on Each Loop to Avoid a Stale Element Exception
    browser.find_by_css("a.product-item h3")[item].click()
    
    # Find Sample Image Anchor Tag & Extract <href>
    sample_element = browser.find_link_by_text("Sample").first
    hemisphere["img_url"] = sample_element["href"]
    
    # Get Hemisphere Title
    hemisphere["title"] = browser.find_by_css("h2.title").text
    
    # Append Hemisphere Object to List
    hemisphere_image_urls.append(hemisphere)
    
    # Navigate Backwards
    browser.back()


In [31]:
for url in hemisphere_image_urls:
    print(url)

{'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg', 'title': 'Cerberus Hemisphere Enhanced'}
{'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg', 'title': 'Schiaparelli Hemisphere Enhanced'}
{'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg', 'title': 'Syrtis Major Hemisphere Enhanced'}
{'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg', 'title': 'Valles Marineris Hemisphere Enhanced'}
